In [1]:
import argparse
from cProfile import label
import os
from PIL import Image
import pickle
import sys

import torch
from transformers import LayoutLMv3FeatureExtractor, LayoutLMv3Tokenizer
from torchvision import transforms
from dall_e.utils import map_pixels
from dall_e import load_model

sys.path.append('../src')
from utils import utils, masking_generator
from utils.slack import notification_slack

/home/is/hikaru-si/.pyenv/versions/3.8.6/envs/exp_005/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
parser = argparse.ArgumentParser()
parser.add_argument("--tokenizer_vocab_dir", type=str)
parser.add_argument("--image_file_dir", type=str, required=True)
parser.add_argument("--pdf_file_dir", type=str, required=True)
parser.add_argument("--output_dir", type=str, required=True)
parser.add_argument("--output_filename", type=str, required=True)
parser.add_argument("--datasize", type=int)
args_list = [
  "--tokenizer_vocab_dir", "../data/vocab/tokenizer_vocab/",
  "--image_file_dir", "../../datasets/pdfs/images/train/",
  "--pdf_file_dir", "../../datasets/pdfs/images/train/",
  "--output_dir", "../data/preprocessing_shared/",
  "--output_filename", "encoded_dataset_mim_30000.pkl",
  "--datasize", "30000",
]
args = parser.parse_args(args_list)


In [7]:
args

Namespace(datasize=50000, image_file_dir='../datasets/pdfs/images/train/', output_dir='./data/preprocessing_shared/', output_filename='encoded_dataset_mim_50000.pkl', pdf_file_dir='../datasets/pdfs/images/train/', tokenizer_vocab_dir='../data/vocab/tokenizer_vocab/')

In [2]:
window_size = (14, 14)
num_masking_patches = 75
max_mask_patches_per_block = None
min_mask_patches_per_block = 16

# generating mask for the corresponding image
mask_generator = masking_generator.MaskingGenerator(
            window_size, num_masking_patches=num_masking_patches,
            max_num_patches=max_mask_patches_per_block,
            min_num_patches=min_mask_patches_per_block,
        )

In [9]:
device = torch.device('cpu')
encoder = load_model("https://cdn.openai.com/dall-e/encoder.pkl", device)
tokenizer = LayoutLMv3Tokenizer(f"{args.tokenizer_vocab_dir}vocab.json", f"{args.tokenizer_vocab_dir}merges.txt")
ids = range(tokenizer.vocab_size)
vocab = tokenizer.convert_ids_to_tokens(ids)

In [13]:
image_names = os.listdir(args.image_file_dir)
if hasattr(args, 'datasize'):
    image_names = image_names[:args.datasize]
file_names = []
for n in image_names:
    name = os.path.splitext(n)[0]
    file_names.append(name)

In [ ]:
len(file_names)

In [ ]:
words, bboxes = utils.extraction_text_from_pdf(args.pdf_file_dir, file_names)